In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torchinfo import summary
import numpy as np
import pandas as pd
from torch.utils.tensorboard import SummaryWriter

import re

from datetime import datetime
import os
import time
import sys
sys.path.append('..')

from utils.functions import *

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

# https://discuss.pytorch.org/t/performance-highly-degraded-when-eval-is-activated-in-the-test-phase/3323/32?page=3

cuda


In [16]:
class MooreNetLayers():
    def __init__(self, num_classes=2):
        self.backbone = [
            nn.Sequential(),
            
            nn.Sequential(
                nn.Sequential(
                    nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=0),
                    nn.ReLU()),        

            ),
            
            nn.Sequential(
                nn.Sequential(
                    nn.Conv2d(32, 64, kernel_size=2, stride=1, padding=0),
                    nn.ReLU()),
                
                nn.Sequential(
                    nn.Conv2d(64, 128, kernel_size=2, stride=1, padding=0),
                    nn.ReLU()),
            ),
        ]

        self.exits = [
            nn.Sequential(
                nn.Flatten(),
                nn.ReLU(),
                nn.Linear(64, num_classes)),
            
            nn.Sequential(
                nn.Flatten(),
                nn.Linear(1152, num_classes)),

            nn.Sequential(
                nn.Flatten(),
                nn.Linear(2048, num_classes))
        ]

class MooreNetWithExits(nn.Module):
    def __init__(self, num_classes=2, exit_loss_weights=[1.0, 0.5, 0.2]):
        super().__init__()

        layers = MooreNetLayers(num_classes=num_classes)

        self.backbone = nn.Sequential(*layers.backbone)
        self.exits    = nn.Sequential(*layers.exits)

        self.exit_threshold = torch.tensor([0.5, 0.7], dtype=torch.float32)
        self.fast_inference_mode = False
        self.measurement_mode = False
        self.exit_loss_weights = exit_loss_weights

    def exit_criterion(self, ee_n, x):
        with torch.no_grad():
            pk = nn.functional.softmax(x, dim=-1)
            nc = torch.max(pk)
            return nc > self.exit_threshold[ee_n]

    def _forward_all_exits(self, x):
        results = []
        for bb, ee in zip(self.backbone, self.exits):
            if self.measurement_mode:
                st = time.process_time()
                x = bb(x)
                im = time.process_time()
                res = ee(x)
                ed = time.process_time()
                results.append([ res, im - st, ed - im ])
            else:
                x = bb(x)
                results.append(ee(x))

        return results

    def forward(self, x):
        if self.fast_inference_mode:
            for ee_n, (bb, ee) in enumerate(zip(self.backbone, self.exits)):
                x = bb(x)
                res = ee(x)
                if self.exit_criterion(ee_n, res):
                    return [res, 'ee' + ee_n]
            return [res, 'main']

        return self._forward_all_exits(x)

    def exits_certainty(self, x):
        results = []
        for ee_n, (bb, ee) in enumerate(zip(self.backbone, self.exits)):
            x = bb(x)
            res = ee(x)
            certainty, predicted = torch.max(nn.functional.softmax(res, dim=-1), 1)
            results.append([ certainty.item(), predicted.item() ])
        return results

    def set_fast_inference_mode(self, mode=True):
        if mode:
            pass
            # self.eval()
        self.fast_inference_mode = mode

    def set_measurement_mode(self, mode=True):
        if mode:
            pass 
            # self.eval()
        self.measurement_mode = mode

In [3]:
model = MooreNetWithExits().to(device)

summary(model, (1, 1, 8, 8))
summary(model)
print(model)

model.load_state_dict(torch.load(f'saves/MooreNetWithExits_save04_2023-05-24-21-00-03'))
model.eval()

MooreNetWithExits(
  (backbone): Sequential(
    (0): Sequential()
    (1): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (2): Sequential(
      (0): Sequential(
        (0): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
  )
  (exits): Sequential(
    (0): Sequential(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): Linear(in_features=64, out_features=2, bias=True)
    )
    (1): Sequential(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): Linear(in_features=115

MooreNetWithExits(
  (backbone): Sequential(
    (0): Sequential()
    (1): Sequential(
      (0): Sequential(
        (0): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
        (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
    (2): Sequential(
      (0): Sequential(
        (0): Conv2d(32, 64, kernel_size=(2, 2), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
      (1): Sequential(
        (0): Conv2d(64, 128, kernel_size=(2, 2), stride=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
      )
    )
  )
  (exits): Sequential(
    (0): Sequential(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): Linear(in_features=64, out_features=2, bias=True)
    )
    (1): Sequential(
      (0): Flatten(start_dim=1, end_dim=-1)
      (1): Linear(in_features=115

In [4]:
year = '2016'
month = '01'
data = CustomMawiDataset(author='MOORE', year=year, month=month, as_matrix=True)

Getting files from ../../datasets/scaled/MOORE/2016/01


torch.Size([514040, 1, 8, 8])
torch.Size([514040])


In [ ]:
(data.labels == 2).sum()

In [6]:
X = data.dataset[0:1].to(device)
y = data.labels[0:1].to(device)
print(X)
print(y)

x = model.backbone[0](X)
print(x)

res = model(X)
print(res)

tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0261, 0.0261, 0.0261],
          [0.0261, 0.0261, 0.0261, 0.0000, 0.0261, 0.0261, 0.0261, 0.0261],
          [0.0261, 0.0261, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
          [0.5515, 0.5515, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]]]],
       device='cuda:0')
tensor([0], device='cuda:0')
tensor([[[[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000],
          [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0261, 0.0261, 0.0261],
          [0.0261, 0.0261, 0.0261, 0.0000, 0.0261, 0.0261, 0.0261, 0.0261],
          [0.0261, 0.0261, 0.000

In [19]:
model = MooreNetWithExits().to(device)

epochs = 20

lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

batch_size = 1000

loader = DataLoader(data, batch_size=batch_size, shuffle=True)

for i in range(epochs):  
    total = 0
    correct = [ 0, 0, 0]
    for b, (X, y) in enumerate(loader):
        X = X.to(device)
        y = y.to(device)
        
        results = model(X)
        
        losses = [ criterion(y_pred, y) for y_pred in results ]
        
        optimizer.zero_grad()        
        for loss in losses[:-1]:
            loss.backward(retain_graph=True)
        losses[-1].backward()
        optimizer.step()        
        
        for exit, y_pred in enumerate(results):
            predicted = torch.max(y_pred.data, 1)[1]
            # print(predicted)

            correct[exit] += (y == predicted).sum()
            
        zeroes = ( y == 0 ).sum()
        ones = ( y == 1 ).sum()
    
        total += len(y)  
        
        acc = ' | '.join([ f'{100*corr/total:.4}% ' for corr in correct ])
        loss_str = ' | '.join([ f'{loss:.4f}' for loss in losses ])
        print(f'Epoch: {i}, Batch: {b}, Zeroes: {zeroes}, Ones: {ones}, {i:03d}: {acc} *** {loss_str}')
        
torch.save(model.state_dict(), f'saves/jupyter_save') 

Epoch: 0, Batch: 0, Zeroes: 516, Ones: 484, 000: 52.0%  | 51.6%  | 48.4%  *** 0.7035 | 0.6976 | 0.6945
Epoch: 0, Batch: 1, Zeroes: 487, Ones: 513, 000: 50.65%  | 49.3%  | 48.95%  *** 0.7069 | 0.6937 | 0.6920
Epoch: 0, Batch: 2, Zeroes: 491, Ones: 509, 000: 50.4%  | 49.53%  | 52.0%  *** 0.7077 | 0.6900 | 0.6895
Epoch: 0, Batch: 3, Zeroes: 491, Ones: 509, 000: 50.23%  | 49.88%  | 53.68%  *** 0.7066 | 0.6874 | 0.6865
Epoch: 0, Batch: 4, Zeroes: 498, Ones: 502, 000: 50.26%  | 51.76%  | 54.72%  *** 0.7043 | 0.6845 | 0.6851
Epoch: 0, Batch: 5, Zeroes: 490, Ones: 510, 000: 50.1%  | 53.28%  | 55.82%  *** 0.7068 | 0.6766 | 0.6798
Epoch: 0, Batch: 6, Zeroes: 510, Ones: 490, 000: 50.51%  | 53.86%  | 56.11%  *** 0.6999 | 0.6759 | 0.6799
Epoch: 0, Batch: 7, Zeroes: 490, Ones: 510, 000: 50.35%  | 55.33%  | 56.51%  *** 0.7043 | 0.6739 | 0.6754
Epoch: 0, Batch: 8, Zeroes: 510, Ones: 490, 000: 49.62%  | 56.46%  | 56.63%  *** 0.6984 | 0.6714 | 0.6731
Epoch: 0, Batch: 9, Zeroes: 488, Ones: 512, 000: 48.8

Epoch: 0, Batch: 87, Zeroes: 481, Ones: 519, 000: 58.2%  | 67.46%  | 66.87%  *** 0.6596 | 0.5689 | 0.5751
Epoch: 0, Batch: 88, Zeroes: 479, Ones: 521, 000: 58.28%  | 67.5%  | 66.9%  *** 0.6543 | 0.5541 | 0.5512
Epoch: 0, Batch: 89, Zeroes: 512, Ones: 488, 000: 58.39%  | 67.54%  | 67.0%  *** 0.6547 | 0.5778 | 0.5675
Epoch: 0, Batch: 90, Zeroes: 533, Ones: 467, 000: 58.49%  | 67.6%  | 66.87%  *** 0.6546 | 0.5698 | 0.5794
Epoch: 0, Batch: 91, Zeroes: 495, Ones: 505, 000: 58.59%  | 67.64%  | 66.96%  *** 0.6503 | 0.5584 | 0.5440
Epoch: 0, Batch: 92, Zeroes: 496, Ones: 504, 000: 58.66%  | 67.64%  | 66.97%  *** 0.6559 | 0.5735 | 0.5663
Epoch: 0, Batch: 93, Zeroes: 533, Ones: 467, 000: 58.75%  | 67.67%  | 67.0%  *** 0.6520 | 0.5574 | 0.5485
Epoch: 0, Batch: 94, Zeroes: 536, Ones: 464, 000: 58.84%  | 67.69%  | 67.02%  *** 0.6562 | 0.5656 | 0.5603
Epoch: 0, Batch: 95, Zeroes: 529, Ones: 471, 000: 58.95%  | 67.75%  | 67.09%  *** 0.6469 | 0.5510 | 0.5335
Epoch: 0, Batch: 96, Zeroes: 463, Ones: 537

Epoch: 0, Batch: 175, Zeroes: 501, Ones: 499, 000: 62.91%  | 69.0%  | 69.43%  *** 0.6403 | 0.5665 | 0.5818
Epoch: 0, Batch: 176, Zeroes: 510, Ones: 490, 000: 62.94%  | 69.02%  | 69.47%  *** 0.6288 | 0.5618 | 0.5356
Epoch: 0, Batch: 177, Zeroes: 505, Ones: 495, 000: 62.97%  | 69.03%  | 69.47%  *** 0.6340 | 0.5655 | 0.5717
Epoch: 0, Batch: 178, Zeroes: 526, Ones: 474, 000: 63.0%  | 69.05%  | 69.47%  *** 0.6377 | 0.5749 | 0.5648
Epoch: 0, Batch: 179, Zeroes: 505, Ones: 495, 000: 63.03%  | 69.06%  | 69.49%  *** 0.6257 | 0.5559 | 0.5351
Epoch: 0, Batch: 180, Zeroes: 499, Ones: 501, 000: 63.06%  | 69.07%  | 69.43%  *** 0.6265 | 0.5520 | 0.5550
Epoch: 0, Batch: 181, Zeroes: 509, Ones: 491, 000: 63.09%  | 69.07%  | 69.47%  *** 0.6278 | 0.5502 | 0.5388
Epoch: 0, Batch: 182, Zeroes: 491, Ones: 509, 000: 63.12%  | 69.08%  | 69.49%  *** 0.6232 | 0.5529 | 0.5346
Epoch: 0, Batch: 183, Zeroes: 492, Ones: 508, 000: 63.15%  | 69.08%  | 69.49%  *** 0.6262 | 0.5457 | 0.5435
Epoch: 0, Batch: 184, Zeroes: 

Epoch: 0, Batch: 260, Zeroes: 474, Ones: 526, 000: 64.89%  | 69.74%  | 70.65%  *** 0.5953 | 0.5160 | 0.5140
Epoch: 0, Batch: 261, Zeroes: 512, Ones: 488, 000: 64.91%  | 69.74%  | 70.68%  *** 0.6085 | 0.5394 | 0.5235
Epoch: 0, Batch: 262, Zeroes: 505, Ones: 495, 000: 64.92%  | 69.74%  | 70.64%  *** 0.6168 | 0.5601 | 0.5377
Epoch: 0, Batch: 263, Zeroes: 462, Ones: 538, 000: 64.93%  | 69.74%  | 70.66%  *** 0.6115 | 0.5347 | 0.4998
Epoch: 0, Batch: 264, Zeroes: 506, Ones: 494, 000: 64.95%  | 69.75%  | 70.67%  *** 0.6033 | 0.5338 | 0.5176
Epoch: 0, Batch: 265, Zeroes: 469, Ones: 531, 000: 64.95%  | 69.75%  | 70.67%  *** 0.6154 | 0.5398 | 0.5114
Epoch: 0, Batch: 266, Zeroes: 514, Ones: 486, 000: 64.97%  | 69.78%  | 70.7%  *** 0.6042 | 0.5289 | 0.5073
Epoch: 0, Batch: 267, Zeroes: 490, Ones: 510, 000: 64.98%  | 69.8%  | 70.72%  *** 0.6165 | 0.5426 | 0.5149
Epoch: 0, Batch: 268, Zeroes: 504, Ones: 496, 000: 65.0%  | 69.81%  | 70.74%  *** 0.6136 | 0.5619 | 0.5285
Epoch: 0, Batch: 269, Zeroes: 4

Epoch: 0, Batch: 351, Zeroes: 486, Ones: 514, 000: 66.0%  | 70.32%  | 72.11%  *** 0.5949 | 0.5211 | 0.4842
Epoch: 0, Batch: 352, Zeroes: 513, Ones: 487, 000: 66.01%  | 70.33%  | 72.12%  *** 0.5829 | 0.5306 | 0.5047
Epoch: 0, Batch: 353, Zeroes: 531, Ones: 469, 000: 66.02%  | 70.35%  | 72.14%  *** 0.6004 | 0.5416 | 0.5065
Epoch: 0, Batch: 354, Zeroes: 503, Ones: 497, 000: 66.03%  | 70.36%  | 72.14%  *** 0.5877 | 0.5259 | 0.4972
Epoch: 0, Batch: 355, Zeroes: 518, Ones: 482, 000: 66.05%  | 70.37%  | 72.16%  *** 0.5807 | 0.5095 | 0.4736
Epoch: 0, Batch: 356, Zeroes: 523, Ones: 477, 000: 66.06%  | 70.38%  | 72.19%  *** 0.5844 | 0.5156 | 0.4911
Epoch: 0, Batch: 357, Zeroes: 500, Ones: 500, 000: 66.07%  | 70.39%  | 72.2%  *** 0.5860 | 0.5275 | 0.4914
Epoch: 0, Batch: 358, Zeroes: 492, Ones: 508, 000: 66.08%  | 70.39%  | 72.21%  *** 0.5951 | 0.5340 | 0.5044
Epoch: 0, Batch: 359, Zeroes: 487, Ones: 513, 000: 66.09%  | 70.39%  | 72.23%  *** 0.5933 | 0.5204 | 0.4744
Epoch: 0, Batch: 360, Zeroes: 

...

Epoch: 19, Batch: 126, Zeroes: 511, Ones: 489, 019: 71.55%  | 80.71%  | 90.58%  *** 0.5683 | 0.4310 | 0.3009
Epoch: 19, Batch: 127, Zeroes: 471, Ones: 529, 019: 71.54%  | 80.72%  | 90.58%  *** 0.5614 | 0.4231 | 0.2916
Epoch: 19, Batch: 128, Zeroes: 504, Ones: 496, 019: 71.55%  | 80.71%  | 90.59%  *** 0.5407 | 0.4221 | 0.2866
Epoch: 19, Batch: 129, Zeroes: 515, Ones: 485, 019: 71.55%  | 80.7%  | 90.58%  *** 0.5364 | 0.4242 | 0.3110
Epoch: 19, Batch: 130, Zeroes: 488, Ones: 512, 019: 71.55%  | 80.69%  | 90.57%  *** 0.5334 | 0.4269 | 0.3002
Epoch: 19, Batch: 131, Zeroes: 487, Ones: 513, 019: 71.52%  | 80.69%  | 90.57%  *** 0.5491 | 0.4168 | 0.2841
Epoch: 19, Batch: 132, Zeroes: 493, Ones: 507, 019: 71.52%  | 80.69%  | 90.57%  *** 0.5496 | 0.4136 | 0.2819
Epoch: 19, Batch: 133, Zeroes: 520, Ones: 480, 019: 71.52%  | 80.68%  | 90.56%  *** 0.5857 | 0.4397 | 0.3006
Epoch: 19, Batch: 134, Zeroes: 493, Ones: 507, 019: 71.51%  | 80.67%  | 90.56%  *** 0.5724 | 0.4344 | 0.3152
Epoch: 19, Batch: 13

Epoch: 19, Batch: 214, Zeroes: 491, Ones: 509, 019: 71.62%  | 80.78%  | 90.49%  *** 0.5778 | 0.4325 | 0.2923
Epoch: 19, Batch: 215, Zeroes: 518, Ones: 482, 019: 71.61%  | 80.77%  | 90.48%  *** 0.5764 | 0.4344 | 0.2951
Epoch: 19, Batch: 216, Zeroes: 532, Ones: 468, 019: 71.61%  | 80.77%  | 90.48%  *** 0.5429 | 0.4136 | 0.2765
Epoch: 19, Batch: 217, Zeroes: 522, Ones: 478, 019: 71.61%  | 80.78%  | 90.48%  *** 0.5395 | 0.4232 | 0.2775
Epoch: 19, Batch: 218, Zeroes: 474, Ones: 526, 019: 71.6%  | 80.77%  | 90.48%  *** 0.5554 | 0.4316 | 0.2936
Epoch: 19, Batch: 219, Zeroes: 525, Ones: 475, 019: 71.6%  | 80.77%  | 90.48%  *** 0.5562 | 0.4387 | 0.2920
Epoch: 19, Batch: 220, Zeroes: 494, Ones: 506, 019: 71.61%  | 80.78%  | 90.49%  *** 0.5071 | 0.3855 | 0.2604
Epoch: 19, Batch: 221, Zeroes: 505, Ones: 495, 019: 71.61%  | 80.79%  | 90.49%  *** 0.5416 | 0.4009 | 0.2606
Epoch: 19, Batch: 222, Zeroes: 497, Ones: 503, 019: 71.61%  | 80.79%  | 90.49%  *** 0.5287 | 0.4058 | 0.2988
Epoch: 19, Batch: 223

Epoch: 19, Batch: 297, Zeroes: 476, Ones: 524, 019: 71.59%  | 80.84%  | 90.54%  *** 0.5644 | 0.4149 | 0.2859
Epoch: 19, Batch: 298, Zeroes: 505, Ones: 495, 019: 71.59%  | 80.84%  | 90.53%  *** 0.5680 | 0.4227 | 0.3136
Epoch: 19, Batch: 299, Zeroes: 478, Ones: 522, 019: 71.58%  | 80.84%  | 90.53%  *** 0.5412 | 0.4234 | 0.2902
Epoch: 19, Batch: 300, Zeroes: 510, Ones: 490, 019: 71.59%  | 80.84%  | 90.53%  *** 0.5456 | 0.4030 | 0.2774
Epoch: 19, Batch: 301, Zeroes: 470, Ones: 530, 019: 71.58%  | 80.83%  | 90.53%  *** 0.5540 | 0.4306 | 0.3076
Epoch: 19, Batch: 302, Zeroes: 495, Ones: 505, 019: 71.58%  | 80.83%  | 90.53%  *** 0.5321 | 0.3971 | 0.2625
Epoch: 19, Batch: 303, Zeroes: 500, Ones: 500, 019: 71.58%  | 80.84%  | 90.53%  *** 0.5371 | 0.4071 | 0.2890
Epoch: 19, Batch: 304, Zeroes: 495, Ones: 505, 019: 71.58%  | 80.84%  | 90.54%  *** 0.5470 | 0.4105 | 0.2698
Epoch: 19, Batch: 305, Zeroes: 466, Ones: 534, 019: 71.58%  | 80.84%  | 90.54%  *** 0.5361 | 0.3986 | 0.2560
Epoch: 19, Batch: 3

Epoch: 19, Batch: 381, Zeroes: 495, Ones: 505, 019: 71.56%  | 80.88%  | 90.56%  *** 0.5341 | 0.4084 | 0.2575
Epoch: 19, Batch: 382, Zeroes: 488, Ones: 512, 019: 71.56%  | 80.88%  | 90.56%  *** 0.5406 | 0.4102 | 0.2774
Epoch: 19, Batch: 383, Zeroes: 508, Ones: 492, 019: 71.56%  | 80.87%  | 90.56%  *** 0.5638 | 0.4414 | 0.2974
Epoch: 19, Batch: 384, Zeroes: 488, Ones: 512, 019: 71.56%  | 80.87%  | 90.56%  *** 0.5696 | 0.4197 | 0.2725
Epoch: 19, Batch: 385, Zeroes: 506, Ones: 494, 019: 71.55%  | 80.87%  | 90.56%  *** 0.5436 | 0.4251 | 0.2829
Epoch: 19, Batch: 386, Zeroes: 503, Ones: 497, 019: 71.56%  | 80.87%  | 90.56%  *** 0.5578 | 0.4234 | 0.2816
Epoch: 19, Batch: 387, Zeroes: 506, Ones: 494, 019: 71.56%  | 80.87%  | 90.56%  *** 0.5362 | 0.4134 | 0.2618
Epoch: 19, Batch: 388, Zeroes: 470, Ones: 530, 019: 71.56%  | 80.87%  | 90.56%  *** 0.5380 | 0.4151 | 0.2899
Epoch: 19, Batch: 389, Zeroes: 496, Ones: 504, 019: 71.56%  | 80.87%  | 90.56%  *** 0.5773 | 0.4059 | 0.2785
Epoch: 19, Batch: 3

Epoch: 19, Batch: 471, Zeroes: 494, Ones: 506, 019: 71.58%  | 80.89%  | 90.61%  *** 0.5424 | 0.4158 | 0.2890
Epoch: 19, Batch: 472, Zeroes: 497, Ones: 503, 019: 71.58%  | 80.88%  | 90.61%  *** 0.5416 | 0.4224 | 0.2733
Epoch: 19, Batch: 473, Zeroes: 491, Ones: 509, 019: 71.58%  | 80.89%  | 90.61%  *** 0.5394 | 0.3880 | 0.2583
Epoch: 19, Batch: 474, Zeroes: 488, Ones: 512, 019: 71.58%  | 80.89%  | 90.61%  *** 0.5296 | 0.3922 | 0.2650
Epoch: 19, Batch: 475, Zeroes: 495, Ones: 505, 019: 71.58%  | 80.88%  | 90.6%  *** 0.5525 | 0.4173 | 0.3127
Epoch: 19, Batch: 476, Zeroes: 505, Ones: 495, 019: 71.58%  | 80.89%  | 90.6%  *** 0.5279 | 0.3959 | 0.2746
Epoch: 19, Batch: 477, Zeroes: 482, Ones: 518, 019: 71.58%  | 80.89%  | 90.6%  *** 0.5321 | 0.4087 | 0.2922
Epoch: 19, Batch: 478, Zeroes: 513, Ones: 487, 019: 71.58%  | 80.89%  | 90.6%  *** 0.5254 | 0.4057 | 0.2955
Epoch: 19, Batch: 479, Zeroes: 510, Ones: 490, 019: 71.58%  | 80.89%  | 90.6%  *** 0.5469 | 0.3883 | 0.2499
Epoch: 19, Batch: 480, Z

In [20]:
model = MooreNetWithExits().to(device)

model.load_state_dict(torch.load(f'saves/jupyter_save'))

model.eval()

correct = [ 0, 0, 0]
total = 0

with torch.no_grad():
    for i in range(510):
        X = data.dataset[i*1000:i*1000+1000].to(device)
        y = data.labels[i*1000:i*1000+1000].to(device)

        results = model(X)
    
        for exit, y_pred in enumerate(results):
            predicted = torch.max(y_pred.data, 1)[1]
            # print(predicted)

            correct[exit] += (y == predicted).sum()
    
        total += len(y)

        acc = ' | '.join([ f'{corr:06d}: {100*corr/total:.4}% ' for corr in correct ])
        print(f'{i:03d}: {acc}')

000: 000942: 94.2%  | 000906: 90.6%  | 000883: 88.3% 
001: 001876: 93.8%  | 001809: 90.45%  | 001770: 88.5% 
002: 002802: 93.4%  | 002689: 89.63%  | 002630: 87.67% 
003: 003740: 93.5%  | 003611: 90.28%  | 003537: 88.43% 
004: 004672: 93.44%  | 004510: 90.2%  | 004426: 88.52% 
005: 005609: 93.48%  | 005413: 90.22%  | 005308: 88.47% 
006: 006550: 93.57%  | 006314: 90.2%  | 006193: 88.47% 
007: 007476: 93.45%  | 007211: 90.14%  | 007077: 88.46% 
008: 008405: 93.39%  | 008102: 90.02%  | 007948: 88.31% 
009: 009326: 93.26%  | 009003: 90.03%  | 008825: 88.25% 
010: 010263: 93.3%  | 009899: 89.99%  | 009699: 88.17% 
011: 010991: 91.59%  | 010751: 89.59%  | 010625: 88.54% 
012: 011563: 88.95%  | 011554: 88.88%  | 011578: 89.06% 
013: 012106: 86.47%  | 012358: 88.27%  | 012534: 89.53% 
014: 012656: 84.37%  | 013154: 87.69%  | 013472: 89.81% 
015: 013209: 82.56%  | 013939: 87.12%  | 014419: 90.12% 
016: 013778: 81.05%  | 014745: 86.74%  | 015369: 90.41% 
017: 014343: 79.68%  | 015549: 86.38%  | 

260: 190412: 72.95%  | 211560: 81.06%  | 233609: 89.51% 
261: 191214: 72.98%  | 212458: 81.09%  | 234523: 89.51% 
262: 191805: 72.93%  | 213379: 81.13%  | 235498: 89.54% 
263: 192399: 72.88%  | 214293: 81.17%  | 236459: 89.57% 
264: 192992: 72.83%  | 215210: 81.21%  | 237423: 89.59% 
265: 193576: 72.77%  | 216133: 81.25%  | 238391: 89.62% 
266: 194167: 72.72%  | 217033: 81.29%  | 239361: 89.65% 
267: 194722: 72.66%  | 217947: 81.32%  | 240319: 89.67% 
268: 195315: 72.61%  | 218854: 81.36%  | 241278: 89.69% 
269: 195916: 72.56%  | 219777: 81.4%  | 242246: 89.72% 
270: 196522: 72.52%  | 220690: 81.44%  | 243221: 89.75% 
271: 197080: 72.46%  | 221593: 81.47%  | 244182: 89.77% 
272: 197679: 72.41%  | 222512: 81.51%  | 245151: 89.8% 
273: 198249: 72.35%  | 223429: 81.54%  | 246120: 89.82% 
274: 198829: 72.3%  | 224345: 81.58%  | 247097: 89.85% 
275: 199407: 72.25%  | 225260: 81.62%  | 248067: 89.88% 
276: 200091: 72.24%  | 226163: 81.65%  | 249021: 89.9% 
277: 201036: 72.32%  | 227077: 81.6

418: 301111: 71.86%  | 338279: 80.73%  | 377060: 89.99% 
419: 301677: 71.83%  | 339205: 80.76%  | 378030: 90.01% 
420: 302238: 71.79%  | 340120: 80.79%  | 378988: 90.02% 
421: 302806: 71.75%  | 341036: 80.81%  | 379953: 90.04% 
422: 303351: 71.71%  | 341952: 80.84%  | 380912: 90.05% 
423: 304071: 71.71%  | 342854: 80.86%  | 381839: 90.06% 
424: 305000: 71.76%  | 343753: 80.88%  | 382718: 90.05% 
425: 305938: 71.82%  | 344676: 80.91%  | 383615: 90.05% 
426: 306878: 71.87%  | 345581: 80.93%  | 384502: 90.05% 
427: 307812: 71.92%  | 346480: 80.95%  | 385386: 90.04% 
428: 308763: 71.97%  | 347395: 80.98%  | 386287: 90.04% 
429: 309674: 72.02%  | 348295: 81.0%  | 387184: 90.04% 
430: 310167: 71.96%  | 349041: 80.98%  | 388165: 90.06% 
431: 310664: 71.91%  | 349798: 80.97%  | 389150: 90.08% 
432: 311158: 71.86%  | 350566: 80.96%  | 390134: 90.1% 
433: 311642: 71.81%  | 351317: 80.95%  | 391119: 90.12% 
434: 312130: 71.75%  | 352086: 80.94%  | 392106: 90.14% 
435: 312598: 71.7%  | 352831: 80.